In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import re
import numpy as np
import pandas as pd
#from chart_studio import plotly as py
#import plotly.figure_factory as ff
from scipy import stats

import gensim
import json


import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

import itertools

import keras

from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, Activation, GlobalAveragePooling1D, Flatten, Concatenate, Conv1D, MaxPooling1D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import concatenate
from keras.optimizers import SGD, RMSprop, Adagrad, Adam
from keras.preprocessing.text import one_hot, text_to_word_sequence, Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils.vis_utils import plot_model

import fnmatch

import warnings

import string
from pathlib import Path
from random import shuffle
from ast import literal_eval

warnings.filterwarnings('ignore')



Using TensorFlow backend.
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [4]:
wordnet_lemmatizer = WordNetLemmatizer()

In [5]:

WINDOWS_SIZE = 10
labels=['none','mild','moderate','moderately severe', 'severe']
num_classes = len(labels)

In [28]:
def transcripts_to_dataframe(directory):
    rows_list = []
        
    filenames = os.listdir(directory)
    
    if ".DS_Store" in filenames:
        filenames.remove(".DS_Store")
        
    for filename in filenames:
        
        transcript_path = os.path.join(directory, filename)
        transcript = pd.read_csv(transcript_path, sep='\t')
        m = re.search("(\d{3})_TRANSCRIPT.csv", filename)
        if m:
            print(filename)
            person_id = m.group(1)
            p = {}
            question = ""
            answer = ""
            lines = len(transcript)
            for i in range(0, lines):
                row = transcript.iloc[i]
                if (row["speaker"] == "Ellie") or (i == lines - 1):
                    p["personId"] = person_id
                    if "(" in str(question):
                        question = question[question.index("(") + 1:question.index(")")]
                    p["question"] = question
                    p["answer"] = answer
                    if question != "":
                        rows_list.append(p)
                    p = {}
                    answer = ""
                    question = row["value"]
                else:
                    answer = str(answer) + " " + str(row["value"])

    all_participants = pd.DataFrame(rows_list, columns=['personId', 'question', 'answer'])
    all_participants.to_csv(directory + 'all.csv', sep=',')
    print("File was created")
    return all_participants

In [29]:
data_path = "/home/nevronas/IG-GPUshare/Amit/transcripts/"
transcripts_to_dataframe(data_path) 
all_participants = pd.read_csv(data_path + 'all.csv', sep=',')

458_TRANSCRIPT.csv
472_TRANSCRIPT.csv
388_TRANSCRIPT.csv
402_TRANSCRIPT.csv
351_TRANSCRIPT.csv
405_TRANSCRIPT.csv
465_TRANSCRIPT.csv
318_TRANSCRIPT.csv
313_TRANSCRIPT.csv
345_TRANSCRIPT.csv
448_TRANSCRIPT.csv
327_TRANSCRIPT.csv
456_TRANSCRIPT.csv
337_TRANSCRIPT.csv
396_TRANSCRIPT.csv
412_TRANSCRIPT.csv
300_TRANSCRIPT.csv
387_TRANSCRIPT.csv
349_TRANSCRIPT.csv
330_TRANSCRIPT.csv
363_TRANSCRIPT.csv
302_TRANSCRIPT.csv
359_TRANSCRIPT.csv
382_TRANSCRIPT.csv
442_TRANSCRIPT.csv
389_TRANSCRIPT.csv
344_TRANSCRIPT.csv
417_TRANSCRIPT.csv
484_TRANSCRIPT.csv
401_TRANSCRIPT.csv
311_TRANSCRIPT.csv
409_TRANSCRIPT.csv
315_TRANSCRIPT.csv
471_TRANSCRIPT.csv
447_TRANSCRIPT.csv
461_TRANSCRIPT.csv
374_TRANSCRIPT.csv
303_TRANSCRIPT.csv
469_TRANSCRIPT.csv
489_TRANSCRIPT.csv
320_TRANSCRIPT.csv
352_TRANSCRIPT.csv
367_TRANSCRIPT.csv
425_TRANSCRIPT.csv
371_TRANSCRIPT.csv
380_TRANSCRIPT.csv
321_TRANSCRIPT.csv
455_TRANSCRIPT.csv
454_TRANSCRIPT.csv
373_TRANSCRIPT.csv
370_TRANSCRIPT.csv
479_TRANSCRIPT.csv
481_TRANSCRI

In [9]:
all_participants.head()

,Unnamed: 0,personId,question,answer
0,0,472,hi i'm ellie thanks for coming in today i was ...,sure
1,1,472,okay,NaN
2,2,472,so how are you doing today,i'm well thank you
3,3,472,that's good,NaN
4,4,472,where are you from originally,southern california


In [10]:
all_participants.columns =  ['index','personId', 'question', 'answer']
all_participants = all_participants.astype({"index": int, "personId": float, "question": str, "answer": str })

In [15]:
def text_to_wordlist(text, remove_stopwords=True, stem_words=False):    
    # Clean the text, with the option to remove stopwords and to stem words.
    
    # Convert words to lower case and split them
    text = text.lower().split()

    # Optionally, remove stop words
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        text = [wordnet_lemmatizer.lemmatize(w) for w in text if not w in stops ]
        text = [w for w in text if w != "nan" ]
    else:
        text = [wordnet_lemmatizer.lemmatize(w) for w in text]
        text = [w for w in text if w != "nan" ]
    
    text = " ".join(text)

    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    
    text = re.sub(r"\<", " ", text)
    text = re.sub(r"\>", " ", text)
    
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # Optionally, shorten words to their stems
    if stem_words:
        text = text.split()
        stemmer = SnowballStemmer('english')
        stemmed_words = [stemmer.stem(word) for word in text]
        text = " ".join(stemmed_words)
    
    # Return a list of words
    return(text)

In [17]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /home/nevronas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nevronas/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [19]:
all_participants_mix = all_participants.copy()
all_participants_mix['answer'] = all_participants_mix.apply(lambda row: text_to_wordlist(row.answer).split(), axis=1)
all_participants_mix.head()

,index,personId,question,answer
0,0,472.0,hi i'm ellie thanks for coming in today i was ...,[sure]
1,1,472.0,okay,[]
2,2,472.0,so how are you doing today,"[i, am, well, thank]"
3,3,472.0,that's good,[]
4,4,472.0,where are you from originally,"[southern, california]"


In [20]:
all_participants_mix_stopwords = all_participants.copy()
all_participants_mix_stopwords['answer'] = all_participants_mix_stopwords.apply(lambda row: text_to_wordlist(row.answer, remove_stopwords=False).split(), axis=1)

In [22]:
words = [w for w in all_participants_mix['answer'].tolist()]
words = set(itertools.chain(*words))
vocab_size = len(words)
print(vocab_size)

7373


In [23]:
words_stop = [w for w in all_participants_mix_stopwords['answer'].tolist()]
words_stop = set(itertools.chain(*words_stop))
vocab_size_stop = len(words_stop)

In [25]:
windows_size = WINDOWS_SIZE
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(all_participants_mix['answer'])
tokenizer.fit_on_sequences(all_participants_mix['answer'])

all_participants_mix['t_answer'] = tokenizer.texts_to_sequences(all_participants_mix['answer'])
all_participants_mix.head()

,index,personId,question,answer,t_answer
0,0,472.0,hi i'm ellie thanks for coming in today i was ...,[sure],[97]
1,1,472.0,okay,[],[]
2,2,472.0,so how are you doing today,"[i, am, well, thank]","[3, 6, 15, 173]"
3,3,472.0,that's good,[],[]
4,4,472.0,where are you from originally,"[southern, california]","[1074, 233]"


In [26]:
windows_size = WINDOWS_SIZE
tokenizer = Tokenizer(num_words=vocab_size_stop)
tokenizer.fit_on_texts(all_participants_mix_stopwords['answer'])
tokenizer.fit_on_sequences(all_participants_mix_stopwords['answer'])

all_participants_mix_stopwords['t_answer'] = tokenizer.texts_to_sequences(all_participants_mix_stopwords['answer'])
all_participants_mix_stopwords.head()

,index,personId,question,answer,t_answer
0,0,472.0,hi i'm ellie thanks for coming in today i was ...,[sure],[163]
1,1,472.0,okay,[],[]
2,2,472.0,so how are you doing today,"[i, am, well, thank, you]","[1, 18, 41, 254, 11]"
3,3,472.0,that's good,[],[]
4,4,472.0,where are you from originally,"[southern, california]","[1167, 319]"


In [27]:

word_index = tokenizer.word_index
word_size = len(word_index)
print(word_index["sad"])

661


In [30]:
windows_size = WINDOWS_SIZE
cont = 0
word_index = tokenizer
phrases_lp_stop = pd.DataFrame(columns=['personId','answer', 't_answer'])
answers = all_participants_mix_stopwords.groupby('personId').agg('sum', axis=1)

print(answers["answer"])

personId
300.0    [good, atlanta, georgia, um, my, parent, are, ...
301.0    [thank, you, mmm, k, i, am, doing, good, thank...
302.0    [i, am, fine, how, about, yourself, i, am, fro...
303.0    [okay, how, bout, yourself, here, in, californ...
304.0    [i, am, doing, good, um, from, los, angeles, c...
305.0    [i, am, doing, alright, uh, originally, i, am,...
306.0    [fine, uh, colorado, mhm, uh, career, career, ...
307.0    [laughter, um, moscow, um, my, family, moved, ...
308.0    [los, angeles, california, yes, um, the, south...
309.0    [laughter, laughter, yeah, laughter, laughter,...
310.0    [yes, it, okay, laughter, fine, laughter, i, l...
311.0    [yes, okay, uh, when, would, i, move, to, l, a...
312.0    [yes, fine, how, about, you, here, yes, the, w...
313.0    [sure, uh, okay, i, guess, not, bad, not, good...
314.0    [yes, quite, well, feel, good, los, angeles, c...
315.0    [alright, yes, okay, and, you, inglewood, cali...
316.0    [yes, i, am, fine, new, york, uh, for,

In [32]:
cont = 0
for p in answers.iterrows():      
    words = p[1]["answer"]
    size = len(words)
    word_tokens = p[1]["t_answer"]
    print(cont)
    for i in range(size):
        sentence = words[i:min(i+windows_size,size)]  
        tokens = word_tokens[i:min(i+windows_size,size)]  
        phrases_lp_stop.loc[cont] = [p[0], sentence, tokens]
        cont = cont + 1

0
366
1910
2537
4564
5586
9031
10645
13317
14253
14978
16167
16840
18094
18864
22270
23681
24333
25004
25734
26438
27229
28181
29783
31179
32089
33961
34490
35308
37387
38393
38893
40421
41348
42739
44279
45715
46737
51500
52373
53416
54025
55336
56086
58111
59615
61954
62412
63425
64967
66450
67651
69384
70677
71189
72243
73920
74343
75023
77376
78019
79352
80184
82410
86641
89041
91691
94841
98498
100818
104009
104853
107126
110843
113307
113865
115659
118143
119495
122094
126155
127879
129032
131212
132345
132527
134731
135297
135775
136363
137814
138437
139156
139911
141173
142312
144098
145299
146517
147752
148927
150409
151498
154051
155105
158365
159245
161725
163222
166294
167179
168929
170431
171685
173201
174465
176141
177836
178954
180049
182566
183609
186170
187978
189206
190309
191330
192564
194061
194949
196252
197200
199869
200911
202344
203590
206969
210093
211990
212964
214603
215140
218037
219080
220873
222305
224942
228109
230186
230789
232313
233009
234107
235798
23

In [33]:
phrases_lp_stop.head()

,personId,answer,t_answer
0,300.0,"[good, atlanta, georgia, um, my, parent, are, ...","[42, 1732, 2098, 5, 12, 205, 37, 69, 113, 5]"
1,300.0,"[atlanta, georgia, um, my, parent, are, from, ...","[1732, 2098, 5, 12, 205, 37, 69, 113, 5, 1]"
2,300.0,"[georgia, um, my, parent, are, from, here, um,...","[2098, 5, 12, 205, 37, 69, 113, 5, 1, 119]"
3,300.0,"[um, my, parent, are, from, here, um, i, love,...","[5, 12, 205, 37, 69, 113, 5, 1, 119, 6]"
4,300.0,"[my, parent, are, from, here, um, i, love, it, i]","[12, 205, 37, 69, 113, 5, 1, 119, 6, 1]"


In [34]:
windows_size = WINDOWS_SIZE
cont = 0
word_index = tokenizer
phrases_lp = pd.DataFrame(columns=['personId','answer', 't_answer'])
answers = all_participants_mix.groupby('personId').agg('sum', axis=1)

for p in answers.iterrows():      
    
    words = p[1]["answer"]
    size = len(words)
    word_tokens = p[1]["t_answer"]
    print(cont)
    for i in range(size):
        sentence = words[i:min(i+windows_size,size)]  
        tokens = word_tokens[i:min(i+windows_size,size)]  
        phrases_lp.loc[cont] = [p[0], sentence, tokens]
        cont = cont + 1

0
216
948
1289
2306
2820
4467
5272
6634
7128
7522
8134
8492
9114
9541
11143
11859
12199
12612
12990
13364
13740
14193
15043
15774
16260
17226
17541
17988
18998
19510
19807
20537
21021
21756
22506
23241
23777
25875
26337
26890
27214
27861
28208
29187
30033
31162
31393
31906
32726
33442
34097
34961
35630
35916
36462
37272
37520
37921
39117
39453
40176
40612
41633
43633
44724
46096
47446
49283
50470
52061
52512
53621
55410
56647
56942
57855
59048
59743
61122
63171
63974
64588
65735
66337
66430
67611
67925
68167
68473
69276
69612
69995
70382
71056
71623
72482
73094
73766
74375
74966
75725
76316
77554
78141
79799
80270
81574
82292
83751
84254
85067
85771
86414
87162
87834
88666
89515
90086
90618
91842
92382
93546
94464
95095
95675
96222
96897
97670
98152
98855
99358
100583
101171
101863
102549
104204
105711
106599
107096
107951
108286
109691
110175
111096
111794
113099
114452
115480
115793
116535
116938
117489
118336
119083
119717
120237
121265
121723
122828
124247
125620
126288
127310
1282

In [35]:


phrases_lp.to_csv(data_path + 'phrases_lp.csv', sep='\t')
print("File was created")

File was created


In [36]:
phrases_lp.head()

,personId,answer,t_answer
0,300.0,"[good, atlanta, georgia, um, parent, um, love,...","[16, 1639, 2007, 1, 131, 1, 63, 5, 142, 5]"
1,300.0,"[atlanta, georgia, um, parent, um, love, like,...","[1639, 2007, 1, 131, 1, 63, 5, 142, 5, 336]"
2,300.0,"[georgia, um, parent, um, love, like, weather,...","[2007, 1, 131, 1, 63, 5, 142, 5, 336, 1]"
3,300.0,"[um, parent, um, love, like, weather, like, op...","[1, 131, 1, 63, 5, 142, 5, 336, 1, 39]"
4,300.0,"[parent, um, love, like, weather, like, opport...","[131, 1, 63, 5, 142, 5, 336, 1, 39, 1]"


In [37]:
phrases_lp["t_answer"] = pad_sequences(phrases_lp["t_answer"], value=0, padding="post", maxlen=windows_size).tolist()
phrases_lp.head()

,personId,answer,t_answer
0,300.0,"[good, atlanta, georgia, um, parent, um, love,...","[16, 1639, 2007, 1, 131, 1, 63, 5, 142, 5]"
1,300.0,"[atlanta, georgia, um, parent, um, love, like,...","[1639, 2007, 1, 131, 1, 63, 5, 142, 5, 336]"
2,300.0,"[georgia, um, parent, um, love, like, weather,...","[2007, 1, 131, 1, 63, 5, 142, 5, 336, 1]"
3,300.0,"[um, parent, um, love, like, weather, like, op...","[1, 131, 1, 63, 5, 142, 5, 336, 1, 39]"
4,300.0,"[parent, um, love, like, weather, like, opport...","[131, 1, 63, 5, 142, 5, 336, 1, 39, 1]"


In [38]:
phrases_lp_stop.to_csv(data_path + 'phrases_lp_stop.csv', sep='\t')
print("File was created")

File was created


In [39]:
phrases_lp = pd.read_csv(data_path + 'phrases_lp.csv', sep='\t', converters={"t_answer": literal_eval}) 
phrases_lp.head()

,Unnamed: 0,personId,answer,t_answer
0,0,300.0,"['good', 'atlanta', 'georgia', 'um', 'parent',...","[16, 1639, 2007, 1, 131, 1, 63, 5, 142, 5]"
1,1,300.0,"['atlanta', 'georgia', 'um', 'parent', 'um', '...","[1639, 2007, 1, 131, 1, 63, 5, 142, 5, 336]"
2,2,300.0,"['georgia', 'um', 'parent', 'um', 'love', 'lik...","[2007, 1, 131, 1, 63, 5, 142, 5, 336, 1]"
3,3,300.0,"['um', 'parent', 'um', 'love', 'like', 'weathe...","[1, 131, 1, 63, 5, 142, 5, 336, 1, 39]"
4,4,300.0,"['parent', 'um', 'love', 'like', 'weather', 'l...","[131, 1, 63, 5, 142, 5, 336, 1, 39, 1]"


In [40]:

def load_avec_dataset_file(path, score_column):
    ds = pd.read_csv(path, sep=',')
    ds['level'] = pd.cut(ds[score_column], bins=[-1,0,5,10,15,25], labels=[0,1,2,3,4])
    ds['PHQ8_Score'] = ds[score_column]
    ds['cat_level'] = keras.utils.to_categorical(ds['level'], num_classes).tolist()
    ds = ds[['Participant_ID', 'level', 'cat_level', 'PHQ8_Score']]
    ds = ds.astype({"Participant_ID": float, "level": int, 'PHQ8_Score': int})
    return ds

In [41]:
train = load_avec_dataset_file('/home/nevronas/IG-GPUshare/Amit/train_split_Depression_AVEC2017.csv', 'PHQ8_Score')
dev = load_avec_dataset_file('/home/nevronas/IG-GPUshare/Amit/dev_split_Depression_AVEC2017.csv', 'PHQ8_Score')
test = load_avec_dataset_file('/home/nevronas/IG-GPUshare/Amit/full_test_split.csv', 'PHQ_Score')
print("Size: train= {}, dev= {}, test= {}".format(len(train), len(dev), len(test)))
train.head()

Size: train= 107, dev= 35, test= 47


,Participant_ID,level,cat_level,PHQ8_Score
0,303.0,0,"[1.0, 0.0, 0.0, 0.0, 0.0]",0
1,304.0,2,"[0.0, 0.0, 1.0, 0.0, 0.0]",6
2,305.0,2,"[0.0, 0.0, 1.0, 0.0, 0.0]",7
3,310.0,1,"[0.0, 1.0, 0.0, 0.0, 0.0]",4
4,312.0,1,"[0.0, 1.0, 0.0, 0.0, 0.0]",2


In [42]:
ds_total = pd.concat([train,dev,test])
total_phq8 = len(ds_total)
print("Total size = {}".format(total_phq8))

Total size = 189


In [43]:
ds_total.to_csv(data_path + 'ds_total.csv', sep = '\t')
print('done')

done


In [44]:
def split_by_phq_level(ds):
    none_ds = ds[ds['level']==0]
    mild_ds = ds[ds['level']==1]
    moderate_ds = ds[ds['level']==2]
    moderate_severe_ds = ds[ds['level']==3]
    severe_ds = ds[ds['level']==4]
    return (none_ds, mild_ds, moderate_ds, moderate_severe_ds, severe_ds)

In [45]:

none_ds, mild_ds, moderate_ds, moderate_severe_ds, severe_ds = split_by_phq_level(ds_total)
print("Quantity per none_ds: {}, mild_ds: {}, moderate_ds {}, moderate_severe_ds: {}, severe_ds {}".format(len(none_ds), len(mild_ds), len(moderate_ds), len(moderate_severe_ds), len(severe_ds)))

Quantity per none_ds: 26, mild_ds: 70, moderate_ds 47, moderate_severe_ds: 24, severe_ds 22


In [46]:
b_none_ds = ds_total[ds_total['level']==0]
b_mild_ds = ds_total[ds_total['level']==1].sample(26)
b_moderate_ds = ds_total[ds_total['level']==2].sample(26)
b_moderate_severe_ds = ds_total[ds_total['level']==3]
b_severe_ds = ds_total[ds_total['level']==4]

ds_total_b = pd.concat([b_none_ds, b_mild_ds, b_moderate_ds, b_moderate_severe_ds, b_severe_ds])
print("Quantity per none_ds: {}, mild_ds: {}, moderate_ds {}, moderate_severe_ds: {}, severe_ds {}".format(len(b_none_ds), len(b_mild_ds), len(b_moderate_ds), len(b_moderate_severe_ds), len(b_severe_ds)))

Quantity per none_ds: 26, mild_ds: 26, moderate_ds 26, moderate_severe_ds: 24, severe_ds 22


In [47]:
ds_lp = pd.merge(ds_total, phrases_lp,left_on='Participant_ID', right_on='personId')
ds_lp_b = pd.merge(ds_total_b, phrases_lp,left_on='Participant_ID', right_on='personId')

In [51]:
def distribute_instances(ds, split_in = [70,14,16]):
    ds_shuffled = ds.sample(frac=1)
    none_ds, mild_ds, moderate_ds, moderate_severe_ds, severe_ds = split_by_phq_level(ds_shuffled)
    eq_ds = dict()
    prev_none = prev_mild = prev_moderate = prev_moderate_severe = prev_severe = 0
    split = split_in
    for p in split:
        last_none = min(len(none_ds), prev_none + round(len(none_ds) * p/100))
        last_mild = min(len(mild_ds), prev_mild + round(len(mild_ds) * p/100))
        last_moderate = min(len(moderate_ds), prev_moderate + round(len(moderate_ds) * p/100))
        last_moderate_severe = min(len(moderate_severe_ds), prev_moderate_severe + round(len(moderate_severe_ds) * p/100))
        last_severe = min(len(severe_ds), prev_severe + round(len(severe_ds) * p/100))  
        eq_ds['d'+str(p)] = pd.concat([none_ds[prev_none: last_none], mild_ds[prev_mild: last_mild], moderate_ds[prev_moderate: last_moderate], moderate_severe_ds[prev_moderate_severe: last_moderate_severe], severe_ds[prev_severe: last_severe]])
        prev_none = last_none
        prev_mild = last_mild
        prev_moderate = last_moderate
        prev_moderate_severe = last_moderate_severe
        prev_severe = last_severe  
    return (eq_ds['d70'], eq_ds['d14'], eq_ds['d16'])

In [52]:
train_lp, dev_lp, test_lp = distribute_instances(ds_lp)
train_lp_b, dev_lp_b, test_lp_b = distribute_instances(ds_lp_b)

In [55]:
train_lp.tail()

,Participant_ID,level,cat_level,PHQ8_Score,Unnamed: 0,personId,answer,t_answer
48688,414.0,4,"[0.0, 0.0, 0.0, 0.0, 1.0]",16,85135,414.0,"['i', 'have', 'read', 'seen', 'picture', 'um',...","[3, 13, 283, 341, 838, 1, 338, 225, 239, 830]"
49179,414.0,4,"[0.0, 0.0, 0.0, 0.0, 1.0]",16,85626,414.0,"['laughter', 'um', 'probably', 'super', 'outgo...","[8, 1, 27, 673, 155, 5, 1156, 639, 328, 30]"
135431,461.0,4,"[0.0, 0.0, 0.0, 0.0, 1.0]",17,120061,461.0,"['well', 'know', 'diagnosis', 'xxx', 'thirty',...","[15, 4, 1574, 152, 477, 22, 58, 152, 477, 213]"
84532,377.0,4,"[0.0, 0.0, 0.0, 0.0, 1.0]",16,58933,377.0,"['supportive', 'loving', 'nurturing', 'um', 'i...","[903, 700, 1956, 1, 3, 6, 100, 91, 3, 6]"
53365,426.0,4,"[0.0, 0.0, 0.0, 0.0, 1.0]",20,94807,426.0,"['age', 'finishing', 'school', 'getting', 'mas...","[467, 1343, 59, 81, 1067, 530, 16, 71, 107, 6213]"


In [56]:
def test_model(text, model):
    print(text)
    word_list = text_to_wordlist(text)
    sequences = tokenizer.texts_to_sequences([word_list])
    sequences_input = list(itertools.chain(*sequences))
    sequences_input =  pad_sequences([sequences_input], value=0, padding="post", maxlen=windows_size).tolist()
    input_a = np.asarray(sequences_input)
    pred = model.predict(input_a, batch_size=None, verbose=0, steps=None)
    predicted_class = np.argmax(pred)
    print(labels[predicted_class])

In [57]:
def confusion_matrix(model, x, y):
    prediction = model.predict(x, batch_size=None, verbose=0, steps=None)
    labels=['none','mild','moderate','moderately severe', 'severe']

    max_prediction = np.argmax(prediction, axis=1)
    max_actual = np.argmax(y, axis=1)

    y_pred = pd.Categorical.from_codes(max_prediction, labels)
    y_actu = pd.Categorical.from_codes(max_actual, labels)

    return pd.crosstab(y_actu, y_pred)

In [58]:
embeddings_index = dict()
f = open('/home/nevronas/IG-GPUshare/Amit/glove.6B.100d.txt', encoding="utf8")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [59]:
def fill_embedding_matrix(tokenizer):
    vocab_size = len(tokenizer.word_index)
    embedding_matrix = np.zeros((vocab_size+1, 100))
    for word, i in tokenizer.word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:        
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [60]:
embedding_matrix_lp = fill_embedding_matrix(tokenizer)

In [68]:
train_lp.head()

,Participant_ID,level,cat_level,PHQ8_Score,Unnamed: 0,personId,answer,t_answer
25947,364.0,0,"[1.0, 0.0, 0.0, 0.0, 0.0]",0,42445,364.0,"['am', 'pretty', 'good', 'think', 'i', 'am', '...","[6, 23, 16, 12, 3, 6, 19, 80, 15, 3]"
740,303.0,0,"[1.0, 0.0, 0.0, 0.0, 0.0]",0,2029,303.0,"['talking', 'wall', 'opposite', 'regretting', ...","[195, 1078, 1042, 2135, 4, 49, 5, 177, 201, 4]"
139588,467.0,0,"[1.0, 0.0, 0.0, 0.0, 0.0]",0,125781,467.0,"['um', 'wanted', 'try', 'something', 'differen...","[1, 125, 62, 29, 38, 29, 3, 10, 85, 110]"
71682,471.0,0,"[1.0, 0.0, 0.0, 0.0, 0.0]",0,129267,471.0,"['diffuses', 'trouble', 'rather', 'prolongs', ...","[4802, 315, 436, 4803, 4804, 315, 2, 40, 43, 10]"
26908,364.0,0,"[1.0, 0.0, 0.0, 0.0, 0.0]",0,43406,364.0,"['gonna', 'nice', 'day', 'looked', 'forward', ...","[104, 129, 42, 800, 524, 96, 905, 2, 36, 3]"


In [95]:
train_lp_copy = train_lp
print(len(train_lp_copy))
for index,row in train_lp_copy.iterrows():
    if len(row['t_answer'])<10:
        train_lp_copy.drop(index,inplace=True)
print(len(train_lp_copy))        

99969
98781


In [96]:
train_lp = train_lp_copy

In [98]:
dev_lp_copy = dev_lp
print(len(dev_lp_copy))
for index,row in dev_lp_copy.iterrows():
    if len(row['t_answer'])<10:
        dev_lp_copy.drop(index,inplace=True)
print(len(dev_lp_copy))  
dev_lp = dev_lp_copy

19993
19767


In [109]:
test_lp_copy = test_lp
print(len(test_lp_copy))
for index,row in test_lp_copy.iterrows():
    if len(row['t_answer'])<10:
        test_lp_copy.drop(index,inplace=True)
print(len(test_lp_copy))  
test_lp = test_lp_copy

22849
22588


In [99]:
train_a = np.stack(train_lp['t_answer'], axis=0)
dev_a = np.stack(dev_lp['t_answer'], axis=0)
train_y = np.stack(train_lp['cat_level'], axis=0)
dev_y = np.stack(dev_lp['cat_level'], axis=0)

In [101]:
train_lp_b_copy = train_lp_b
print(len(train_lp_b_copy))
for index,row in train_lp_b_copy.iterrows():
    if len(row['t_answer'])<10:
        train_lp_b_copy.drop(index,inplace=True)
print(len(train_lp_b_copy))  
train_lp_b = train_lp_b_copy

65594
64835


In [102]:
dev_lp_b_copy = dev_lp_b
print(len(dev_lp_b_copy))
for index,row in dev_lp_b_copy.iterrows():
    if len(row['t_answer'])<10:
        dev_lp_b_copy.drop(index,inplace=True)
print(len(dev_lp_b_copy))  
dev_lp_b = dev_lp_b_copy

13118
12963


In [110]:
test_lp_b_copy = test_lp_b
print(len(test_lp_b_copy))
for index,row in test_lp_b_copy.iterrows():
    if len(row['t_answer'])<10:
        test_lp_b_copy.drop(index,inplace=True)
print(len(test_lp_b_copy))  
test_lp_b = test_lp_b_copy

14992
14808


In [103]:

train_a_b = np.stack(train_lp_b['t_answer'], axis=0)
dev_a_b = np.stack(dev_lp_b['t_answer'], axis=0)
train_y_b = np.stack(train_lp_b['cat_level'], axis=0)
dev_y_b = np.stack(dev_lp_b['cat_level'], axis=0)

In [104]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3)

In [105]:

answer_inp = Input(shape=(windows_size, ))
embedding_size_glove = 100
answer_emb1 = Embedding(vocab_size_stop+1, embedding_size_glove, weights=[embedding_matrix_lp], input_length=windows_size, trainable=False)(answer_inp)

bt = BatchNormalization()(answer_emb1)
lstm = LSTM(embedding_size_glove, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(bt)

dense1 = Dense(units=256, activation="relu")(lstm)
dense2 = Dense(units=256, activation="relu")(dense1)

flatten = Flatten()(dense2)

out = Dense(5,  activation='softmax')(flatten)

model = Model(inputs=[answer_inp], outputs=[out])
model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 10, 100)           745000    
_________________________________________________________________
batch_normalization_1 (Batch (None, 10, 100)           400       
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 100)           80400     
_________________________________________________________________
dense_1 (Dense)              (None, 10, 256)           25856     
_________________________________________________________________
dense_2 (Dense)              (None, 10, 256)           65792     
_________________________________________________________________
flatten_1 (Flatten)          (None, 2560)              0   

In [106]:
model_glove_lstm_hist = model.fit(train_a, train_y, validation_data=(dev_a, dev_y), epochs=30, batch_size=64, shuffle=True, callbacks=[early_stopping])

Train on 98781 samples, validate on 19767 samples
Epoch 1/30
98781/98781 [==============================] - 20s 201us/step - loss: 1.4541 - accuracy: 0.3723 - val_loss: 1.3433 - val_accuracy: 0.4316
Epoch 2/30
98781/98781 [==============================] - 18s 182us/step - loss: 1.2717 - accuracy: 0.4693 - val_loss: 1.0919 - val_accuracy: 0.5693
Epoch 3/30
98781/98781 [==============================] - 19s 189us/step - loss: 1.1037 - accuracy: 0.5548 - val_loss: 0.9078 - val_accuracy: 0.6512
Epoch 4/30
98781/98781 [==============================] - 17s 177us/step - loss: 0.9968 - accuracy: 0.6030 - val_loss: 0.7884 - val_accuracy: 0.6960
Epoch 5/30
98781/98781 [==============================] - 18s 185us/step - loss: 0.9143 - accuracy: 0.6381 - val_loss: 0.7090 - val_accuracy: 0.7345
Epoch 6/30
98781/98781 [==============================] - 18s 186us/step - loss: 0.8538 - accuracy: 0.6662 - val_loss: 0.6143 - val_accuracy: 0.7778
Epoch 7/30
98781/98781 [==============================] 

In [107]:
model_glove_lstm_hist_b = model.fit(train_a_b, train_y_b, validation_data=(dev_a_b, dev_y_b), epochs=30, batch_size=64, shuffle=True, callbacks=[early_stopping])

Train on 64835 samples, validate on 12963 samples
Epoch 1/30
64835/64835 [==============================] - 12s 188us/step - loss: 0.3955 - accuracy: 0.8564 - val_loss: 0.0779 - val_accuracy: 0.9787
Epoch 2/30
64835/64835 [==============================] - 11s 177us/step - loss: 0.3508 - accuracy: 0.8714 - val_loss: 0.0765 - val_accuracy: 0.9793
Epoch 3/30
64835/64835 [==============================] - 11s 173us/step - loss: 0.3235 - accuracy: 0.8823 - val_loss: 0.0658 - val_accuracy: 0.9818
Epoch 4/30
64835/64835 [==============================] - 11s 176us/step - loss: 0.3071 - accuracy: 0.8883 - val_loss: 0.0615 - val_accuracy: 0.9845
Epoch 5/30
64835/64835 [==============================] - 11s 176us/step - loss: 0.3020 - accuracy: 0.8915 - val_loss: 0.0607 - val_accuracy: 0.9843
Epoch 6/30
64835/64835 [==============================] - 11s 177us/step - loss: 0.2879 - accuracy: 0.8962 - val_loss: 0.0631 - val_accuracy: 0.9832
Epoch 7/30
64835/64835 [==============================] 

In [111]:
test_a = np.stack(test_lp['t_answer'], axis=0)
test_y = np.stack(test_lp['cat_level'], axis=0)
test_a_b = np.stack(test_lp_b['t_answer'], axis=0)
test_y_b = np.stack(test_lp_b['cat_level'], axis=0)
df_confusion = confusion_matrix(model, test_a_b, test_y_b)

In [112]:
df_confusion

col_0,none,mild,moderate,moderately severe,severe
row_0,,,,,
none,3520,14,10,1,6
mild,7,2504,27,6,3
moderate,10,7,3493,3,0
moderately severe,19,11,25,2745,10
severe,10,8,15,4,2350


In [113]:
score = model.evaluate(test_a_b, test_y_b, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.047430607787097456
Test accuracy: 0.9867638945579529


In [130]:
model.save('/home/nevronas/IG-GPUshare/Amit/models/model_glove_lstm_b.h5')
json_dict = model_glove_lstm_hist.history
with open('/home/nevronas/IG-GPUshare/Amit/models/model_glove_lstm_b_hist.json', 'w') as f:
    f.write(str(json_dict))
print('File got created')    

File got created
